In [ ]:
from SCPDefinitions import *
from SCPConstructive import *
from SCPLocalSearch import *
from SCP_solve_all import *
from SCP_GRASP import *
from SCP_TABU import *

# Instance Representation — `SCPInstance`

Given an SCP instance:

- $m$: number of attributes (elements to cover)  
- $n$: number of sets (airplanes)  
- $\mathbf{c} = [c_1, c_2, \dots, c_n]$: cost of each airplane  
- Coverage is stored implicitly using sparse mappings:

  - `attr_of_set[j]` → attributes covered by set \( j \)  
    (corresponds to column \( j \) of \( A \))  

  - `sets_of_attr[i]` → sets that cover attribute \( i \)  
    (corresponds to row \( i \) of \( A \))

Thus,

$$
a_{ij} = 1 \iff i \in \text{attr\_of\_set}[j] \iff j \in \text{sets\_of\_attr}[i].
$$

The instance defines the data for the optimization model but does not explicitly store matrix \(A\).


In [ ]:
inst = SCPInstance(3)
inst.summary()

# Solution representation

A solution is represented by a binary vector of size n, where each position i indicates whether subset i is included in the cover (1) or not (0).

Given an SCP instance defined by:

- $m$: number of attributes (elements to cover)  
- $n$: number of sets (airplanes)  
- $\mathbf{c} = [c_1, c_2, \dots, c_n]$: cost vector  
- $A \in \{0,1\}^{m \times n}$: coverage matrix with entries  
  $$
  a_{ij} =
  \begin{cases}
  1, & \text{if attribute } i \text{ is covered by set } j, \\
  0, & \text{otherwise.}
  \end{cases}
  $$

---

### **Decision Variables**

Each airplane (set) $j$ has a binary decision variable:
$$
x_j =
\begin{cases}
1, & \text{if airplane } j \text{ is selected,} \\
0, & \text{otherwise.}
\end{cases}
$$

In code:
- `self.selected`  →  $\{\,j : x_j = 1\,\}$  
- `self.covered[i]`  →  $\displaystyle \sum_{j=1}^{n} a_{ij} x_j$  
- `self.cost`  →  $\displaystyle \sum_{j=1}^{n} c_j x_j$

---

### **Objective Function**

Minimize the total cost of selected airplanes:
$$
\min_{x \in \{0,1\}^n} Z = \sum_{j=1}^{n} c_j x_j
$$

---

### **Feasibility Constraints**

Each attribute must be covered by at least one selected set:
$$
\sum_{j=1}^{n} a_{ij} x_j \ge 1, \quad \forall\, i = 1,\dots,m.
$$

In code:  
`is_feasible()` → checks `np.all(self.covered > 0)`

---




In [ ]:
inst = SCPInstance(3)
sol = SCPSolution(inst)

sol.add(5)
sol.add(10)
sol.add(200)
sol.remove(10)

sol.summary()
sol.prune_by_cost()
sol.summary()


In [ ]:
#Load from csv
inst = SCPInstance(0, folder="SCP-Instances")
sol = SCPSolution.from_csv(inst, "results/greedy_RE.csv")

sol.summary()


# CONSTRUCTION HEURISTIC

In [ ]:
from SCPConstructive import *

### ✈️ CH0: Greedy First Fit Heuristic

<span style="color:red">Not being used</span>

A simple **constructive heuristic** that builds a feasible solution step by step.

**Idea:**  
Start with no airplanes selected. Repeatedly pick the first uncovered attribute \( i \),  
choose the first set \( j \) that covers it, set \( x_j \leftarrow 1 \),  
update coverage, and stop when all attributes are covered.

Formally, at each iteration:

$$
j^{*} = \min \{\, j : a_{ij} = 1 \,\}, \qquad
i = \text{first uncovered attribute}.
$$

Then set \( x_{j^{*}} = 1 \).

**Characteristics:**  
- Deterministic  
- Greedy and myopic (covers one attribute at a time)  
- Produces a feasible solution, not necessarily optimal  
- A pruning step can later remove redundant sets


In [ ]:
inst = SCPInstance(0)  # load first file in folder
sol = greedy_first_fit(inst)
sol.summary()
sol.prune_by_cost()
sol.summary()


### 💰 Ch1 : Greedy Cost-Efficient Heuristic

A **cost-aware constructive heuristic** that builds a feasible solution by balancing coverage and cost.

**Idea:**  
At each step, select the airplane \( j \) that minimizes the ratio between its cost and  
the number of *new* uncovered attributes it would cover.

Formally:

$$
h(j) = \frac{c_j}{|\{\, i : a_{ij} = 1 \text{ and } \text{covered}[i] = 0 \,\}|}
$$

At each iteration:

$$
j^{*} = \arg\min_{j \notin \text{selected}} h(j)
\quad \Longrightarrow \quad x_{j^{*}} = 1.
$$

Repeat until all attributes are covered.

**Characteristics:**  
- Deterministic  
- Balances cost vs. coverage gain  
- More informed than pure greedy-first-fit  
- Still locally myopic — does not look ahead


In [ ]:
inst = SCPInstance(0)
sol = greedy_cost_efficiency(inst)
sol.summary()
sol.prune_by_cost()
sol.summary()


### 💰 CH2: Greedy Cost Squared over Cover Heuristic

A **cost-sensitive constructive heuristic** that increases the penalty on expensive sets while still favoring wide coverage.

**Idea:**  
At each step, select the airplane \( j \) that minimizes the ratio between the **square of its cost** and  
the number of *new* uncovered attributes it would cover.

Formally:

$$
h(j) = \frac{c_j^2}{|\{\, i : a_{ij} = 1 \text{ and } \text{covered}[i] = 0 \,\}|}
$$

At each iteration:

$$
j^{*} = \arg\min_{j \notin \text{selected}} h(j)
\quad \Longrightarrow \quad x_{j^{*}} = 1.
$$

Repeat until all attributes are covered.

**Characteristics:**  
- Deterministic  
- Applies a quadratic cost penalty (discourages expensive sets)  
- Favors multiple cheap sets over single costly ones  
- More conservative than the standard cost-efficiency heuristic  
- Still locally greedy — no lookahead or randomness


In [ ]:
inst = SCPInstance(0)
sol = greedy_cost_square_over_cover(inst)
sol.summary()
sol.prune_by_cost()
sol.summary()


### 🎲 CH3: Greedy Randomized Adaptive Heuristic (GRASP Constructive Phase)

A **stochastic extension** of the greedy cost-efficient heuristic.

At each step:
- Compute the efficiency of each set  
  \( h(j) = \dfrac{c_j}{|\{\, i : a_{ij} = 1 \text{ and } \text{covered}[i] = 0 \,\}|} \)
- Sort sets by efficiency (lower is better)
- Build a **Restricted Candidate List (RCL)** containing the best candidates:
  $$
  \text{RCL} = \{\, j : h(j) \le h_{\min} + \alpha (h_{\max} - h_{\min}) \,\}
  $$
- Randomly pick one set \( j^* \in \text{RCL} \)
- Add that set to the solution: \( x_{j^*} \leftarrow 1 \)
- Update covered attributes

Repeat until all attributes are covered.

**Parameter:**
- \( \alpha \in [0,1] \): controls greediness  
  - \( \alpha = 0 \): purely greedy  
  - \( \alpha = 1 \): purely random  

**Characteristics:**  
- Randomized but adaptive to current coverage  
- Balances exploration (randomness) and exploitation (greedy choice)  
- Provides diverse starting solutions for local search


In [ ]:
inst = SCPInstance(0)
sol = greedy_randomized_adaptive(inst, alpha=0.1)
sol.summary()
sol.prune_by_cost()
sol.summary()


# Look ahead heuristics
<span style="color:red">Not being used</span>

Non-myopic heuristics that consider future consequences of current choices.

\textcolor{orange}{To Be improved}

In [ ]:
df_greedy = solve_all_instances("greedy_cost_efficiency", "greedy.csv")
df_greedy_RE = solve_all_instances("greedy_RE", "greedy_RE.csv") 

In [ ]:
df_squared = solve_all_instances("greedy_cost_square_over_cover", "greedy_squared_results.csv")
df_squared_RE = solve_all_instances("squared_RE", "squared_RE_results.csv")

In [ ]:
df_random = solve_all_instances("greedy_randomized_adaptive", "random.csv", alpha=0.0)
df_random_RE = solve_all_instances("randomized_RE", "random_RE.csv", alpha=0.0)

# Local Search 

Tasks:

Consider one neighbourhood

Implement first-improvement (FI) and best-improvement (BI) algorithms for the SCP. 
 
In these two algorithms, consider one neighborhoods of your choice. Apply redundancy elimination after each step.

Apply each of these algorithms once to an initial solution generated by CH1, CH2, CH3, and CH1+RE.  H

ence, in total eight algorithms should be tested obtained by the combinations of the four constructive heuristics with the two iterative improvement algorithms. 

As variance reduction technique for the experiments make sure that the FI and BI algorithms start from the same initial solution. This can be ensured by using for each of the executions on a same instance the same random number seed. 

As the experimental results report for each of the experiments
the average percentage deviation from best known solutions;
the total computation time across all instances;
the fraction of instances that profit from the additional local search phase.

Determine by means of statistical tests (in this case, the Student t-test or the Wilcoxon test), whether there is a statistically significant difference between the solutions generated by the various algorithms. Consider the data for the Set Covering Problem (SCP) available in Moodle.


# Best Improvement Local Search 1x1

# First Improvement Local Search 1x1

In [ ]:
#example
inst = SCPInstance(0)
#sol.summary()

print("Squared + RE")
sol_squared_RE = greedy_cost_square_over_cover(inst)
sol_squared_RE = sol.copy()
sol_squared_RE.prune_by_cost()
sol_squared_RE.summary()

print("Squared + FI + RE")
sol_squared_FI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_FI1x1 = best_improvement_1x1_loop(sol, max_time=99999)
sol_squared_FI1x1.prune_by_cost()
sol_squared_FI1x1.summary()

print("Squared + BI + RE")
sol_squared_BI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_BI1x1 = best_improvement_1x1_loop(sol, max_time=99999)
sol_squared_BI1x1.prune_by_cost()
sol_squared_BI1x1.summary()



# Running all instances for 1x1 Search

In [ ]:
df_greedy_RE = solve_all_instances("greedy_RE", "greedy_RE.csv")
df_greedy_FI_RE = solve_all_instances("greedy_plus_FI1x1", "greedy_FI1x1.csv", fi_time=9999.0)
df_greedy_BI_RE = solve_all_instances("greedy_plus_BI1x1", "greedy_BI1x1.csv", ls_time=9999.0)



In [ ]:
df_squared_RE = solve_all_instances("squared_plus_RE", "squared_RE.csv")
df_greedy_FI_RE = solve_all_instances("squared_plus_FI1x1", "squared_FI1x1.csv", fi_time=9999.0)
df_greedy_BI_RE = solve_all_instances("squared_plus_BI1x1", "squared_BI1x1.csv", ls_time=9999.0)



# Best Improvement Local Search Drop or Swap

# First Improvement Local Search Drop or Swap

In [ ]:
#example
inst = SCPInstance(0)
#sol.summary()

print("Squared + RE")
sol = greedy_cost_square_over_cover(inst)
sol_squared_RE = sol.copy()
sol_squared_RE.prune_by_cost()
sol_squared_RE.summary()

print("Squared + FI + RE")
sol_squared_FI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_FI1x1 = first_improvement_drop_or_swap_loop(sol, max_time=99999)
sol_squared_FI1x1.prune_by_cost()
sol_squared_FI1x1.summary()

print("Squared + BI + RE")
sol_squared_BI1x1 = greedy_cost_square_over_cover(inst)
sol_squared_BI1x1 = best_improvement_drop_or_swap_loop(sol, max_time=99999)
sol_squared_BI1x1.prune_by_cost()
sol_squared_BI1x1.summary()



# Running All Instances for Drop or Swap

In [ ]:
df_greedy_RE = solve_all_instances("greedy_plus_RE", "greedy_RE.csv")
df_greedy_FI_RE = solve_all_instances("greedy_plus_FI_drop_or_swap", "greedy_FI_drop_or_swap.csv", fi_time=9999.0)
df_greedy_BI_RE = solve_all_instances("greedy_plus_BI_drop_or_swap", "greedy_BI_drop_or_swap.csv", ls_time=9999.0)



In [ ]:
df_squared_RE = solve_all_instances("squared_plus_RE", "squared_RE.csv")
df_squared_FI_RE = solve_all_instances("squared_plus_FI_drop_or_swap", "squared_FI_drop_or_swap.csv", fi_time=9999.0)
df_squared_BI_RE = solve_all_instances("squared_plus_BI_drop_or_swap", "squared_BI_drop_or_swap.csv", ls_time=9999.0)

In [ ]:
#randomized
df_randomized_RE = solve_all_instances("randomized_plus_RE", "randomized_RE.csv")
df_randomized_FI_RE = solve_all_instances("randomized_plus_FI_drop_or_swap", "randomized_FI_drop_or_swap.csv", alpha = 0.0, fi_time=9999.0)
df_randomized_BI_RE = solve_all_instances("randomized_plus_BI_drop_or_swap", "randomized_BI_drop_or_swap.csv", alpha = 0.0, ls_time=9999.0)

In [ ]:
df_greedy_RE_FI = solve_all_instances("greedy_RE_plus_FI_drop_or_swap", "greedy_RE_FI_drop_or_swap.csv", fi_time=9999.0)
df_greedy_RE_BI = solve_all_instances("greedy_RE_plus_BI_drop_or_swap", "greedy_RE_BI_drop_or_swap.csv", ls_time=9999.0)

# Pruning First

In [ ]:
def greedy_RE_BI1X1(instance, ls_time=999.0):
    sol = greedy_cost_efficiency(instance)
    sol.prune_by_cost()
    sol = best_improvement_1x1_loop(sol, max_time=ls_time)
    return sol

def greedy_plus_FI1x1(instance, fi_time=999.0):
    sol = greedy_cost_efficiency(instance)
    sol.prune_by_cost()
    sol = first_improvement_1x1_loop(sol, max_time=fi_time)
    return sol

df_greedy_RE_BI1X1 = solve_all_instances("greedy_RE_BI1X1", "greedy_RE_BI1X1_results.csv", ls_time=9999.0)
df_greedy_RE_FI1X1 = solve_all_instances("greedy_RE_FI1X1", "greedy_RE_FI1X1_results.csv", fi_time=9999.0)

## Comparing all Local Searches

In [13]:
%load_ext autoreload
%autoreload 2
from SCP_solve_all import solve_all_instances_parallel
from SCPLocalSearch import *


#! Swap or Drop
df_greedy_BI_drop_or_swap = solve_all_instances_parallel(
    "greedy_plus_BI_drop_or_swap", 
    "greedy_RE_BI_drop_or_swap_results_parallel.csv", 
    ls_time=9999.0
    )
df_greedy_FI_drop_or_swap = solve_all_instances_parallel(
    "greedy_plus_FI_drop_or_swap", 
    "greedy_RE_FI_drop_or_swap_results_parallel.csv", 
    fi_time=9999.0
    )

#! RE then Swap or Drop
df_greedy_RE_BI_drop_or_swap = solve_all_instances_parallel(
    "greedy_RE_plus_BI_drop_or_swap", 
    "greedy_RE_BI_drop_or_swap_results_parallel.csv", 
    ls_time=9999.0
    )
df_greedy_RE_FI_drop_or_swap = solve_all_instances_parallel(
    "greedy_RE_plus_FI_drop_or_swap", 
    "greedy_RE_FI_drop_or_swap_results_parallel.csv", 
    fi_time=9999.0
    )



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Solver (parallel): greedy_plus_BI_drop_or_swap using 4 workers


KeyboardInterrupt: 

In [ ]:

#! 1x1 then RE
df_greedy_plus_BI1x1 = solve_all_instances_parallel(
    "greedy_plus_BI1x1", 
    "greedy_plus_BI1x1_results_parallel.csv", 
    ls_time=9999.0
    )
df_greedy_plus_FI1x1_RE = solve_all_instances_parallel(
    "greedy_plus_FI1x1_RE",
    "greedy_plus_FI1x1_RE_results_parallel.csv",
    fi_time=9999.0
    )

#! RE then 1x1
df_greedy_RE_BI1X1 = solve_all_instances_parallel(
    "greedy_RE_BI1X1", 
    "greedy_RE_BI1X1_results_parallel.csv", 
    ls_time=9999.0
    )
df_greedy_RE_FI1x1 = solve_all_instances_parallel(
    "greedy_RE_FI1x1", 
    "greedy_RE_FI1x1_results_parallel.csv", 
    fi_time=9999.0
    )

# GRASP : Greedy Randomized Adaptive Search Procedure

## Sequential Execution of GRASP

In [ ]:
from SCPDefinitions import *
from SCPConstructive import *
from SCPLocalSearch import *
from SCP_solve_all import *
from SCP_GRASP import *

In [ ]:


small_instance = SCPInstance(0)
large_instance = SCPInstance(33)  # adjust index as needed

random.seed(42)
small = grasp_sequential(small_instance, alpha=0.05, max_time=300.0, max_iter=10, verbose=True)
#large = grasp_sequential(large_instance, alpha=0.05, max_time=300.0, max_iter=10, verbose=True)

small.summary()
#large.summary()


## GRASP Parallel Execution

In [ ]:
small_instance = SCPInstance(0)
large_instance = SCPInstance(33)  # adjust index as needed

small = grasp_parallel(small_instance, alpha=0.05, max_time=300.0, num_iter=10)
large = grasp_parallel(large_instance, alpha=0.05, max_time=300.0, num_iter=10)

small.summary()
large.summary()


# GRASP : Adapting Parameters
## Constant RCL Size

In [ ]:

#small_instance.summary()
#large_instance.summary()    

#grasp_parallel_fixed_RCL(large_instance, desired_RCL=20, max_time=600.0, num_iter=10, num_workers=6, seed=42)
grasp_parallel_fixed_RCL(small_instance, desired_RCL=20, max_time=600.0, num_iter=100, num_workers=6, seed=42)

In [ ]:
solve_all_instances(
    "grasp_parallel_fixed_RCL", 
    "grasp_parallel_fixed_RCL.csv", 
    num_instances=0, 
    desired_RCL=20, 
    max_time=600.0, 
    num_iter=100, 
    num_workers=6, 
    seed=42)


solve_all_instances(
    "grasp_parallel",
    "grasp_parallel_results.csv",
    num_instances=0,
    alpha=0.05,
    max_time=600.0,
    num_iter=100,
    num_workers=6,
    seed=42)




## Comparing GRASP RCL fixed FI vs BI

In [ ]:
import importlib
import SCP_GRASP
importlib.reload(SCP_GRASP)
from SCP_GRASP import *

grasp_parallel_BI_fixed_RCL(large_instance, desired_RCL=20, max_time=600.0, num_iter=10, num_workers=6, seed=42)
grasp_parallel_fixed_RCL(large_instance, desired_RCL=20, max_time=600.0, num_iter=10, num_workers=6, seed=42)

In [ ]:
import importlib
import SCP_GRASP
importlib.reload(SCP_GRASP)
from SCP_GRASP import *
import SCP_solve_all
importlib.reload(SCP_solve_all)
from SCP_solve_all import *


solve_all_instances(
    "grasp_parallel_BI_fixed_RCL", 
    "grasp_parallel_BI_fixed_RCL.csv", 
    num_instances=0, 
    desired_RCL=20, 
    max_time=600.0, 
    num_iter=100, 
    num_workers=6, 
    seed=42)


solve_all_instances(
    "grasp_parallel_fixed_RCL",
    "grasp_parallel_fixed_RCL.csv",
    num_instances=0,
    alpha=0.05,
    max_time=600.0,
    num_iter=100,
    num_workers=6,
    seed=42)





In [ ]:
solve_all_instances(
    greedy_RE
    , "greedy_RE.csv"
)

In [ ]:
import SCP_solve_all
importlib.reload(SCP_solve_all)
from SCP_solve_all import *

solve_all_instances_parallel(
    greedy_RE
    , "greedy_RE_parallel.csv"
    , num_workers=10
)

In [ ]:
import importlib
import SCPConstructive
importlib.reload(SCPConstructive)
from SCPConstructive import *

import importlib
import SCPLocalSearch
importlib.reload(SCPLocalSearch)
from SCPLocalSearch import *

importlib.reload(SCP_solve_all)
from SCP_solve_all import *
import SCP_GRASP
importlib.reload(SCP_GRASP)

from SCP_GRASP import *
import SCP_solve_all
importlib.reload(SCP_solve_all)
from SCP_solve_all import *

solve_all_instances(greedy_cost_square_over_cover, "squared_RE_results_sequential.csv")



# TABU Search

In [ ]:
import importlib
import SCP_TABU
importlib.reload(SCP_TABU)
from SCP_TABU import *

small_instance = SCPInstance(0)
large_instance = SCPInstance(33) 

tabu_small = tabu(small_instance, max_time=300.0, tabu_tenure=50, max_no_improve=1000)
tabu_large = tabu(large_instance, max_time=300.0, tabu_tenure=50, max_no_improve=1000)